In [36]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import time

import psycopg2
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

from deepface import DeepFace
from deepface.commons import functions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

Base = declarative_base()

# create an engine to connect to the PostgreSQL database
engine = create_engine('postgresql://postgres:@localhost:5430/faceio')

In [37]:
class FaceMeta(Base):
    __tablename__ = 'face_metas'

    id = Column(Integer, primary_key=True)
    empl_no = Column(Integer, nullable=False)
    img_name = Column(String, nullable=False)
    embedding = Column(LargeBinary, nullable=False)

class FaceEmbedding(Base):
    __tablename__ = 'face_embedding'

    id = Column(Integer, primary_key=True)
    face_id = Column(Integer, nullable=False)
    dimension = Column(Integer, nullable=False)
    value = Column(Float, nullable=False)


In [38]:
from collections import defaultdict
import tensorflow as tf

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()
model = DeepFace.build_model("Facenet")

facial_img_paths = []

for root, directory, files in os.walk("/Users/ds_learner16/Documents/Face.io/deepface"):
    for file in files:
        if '.jpg' in file:
            facial_img_paths.append(root+"/"+file)

# create dictionary to group images by employee
employee_images = defaultdict(list)
for facial_img_path in facial_img_paths:
    employee_no = facial_img_path.split("/")[-2] # assuming employee number is in the second-to-last directory name
    employee_images[employee_no].append(facial_img_path)

instances = []
for employee_no, facial_img_paths in employee_images.items():
    employee_embeddings = []

    for facial_img_path in tqdm(facial_img_paths, desc=f"Processing employee {employee_no}"):
        # load image and preprocess
        img = tf.keras.preprocessing.image.load_img(facial_img_path, target_size=(160, 160))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # represent
        embedding = model.predict(x)[0]
        employee_embeddings.append(embedding)

        # store face embedding
        instance = []
        instance.append(facial_img_path)
        instance.append(embedding)
        instances.append(instance)

    # store employee embeddings
    employee_embeddings = np.array(employee_embeddings)
    face_meta = FaceMeta(empl_no=employee_no, img_name="", embedding=employee_embeddings.tobytes())
    session.add(face_meta)

session.commit()

# store individual face embeddings
for index, instance in tqdm(enumerate(instances), total=len(instances)):
    img_name = instance[0]
    embeddings = instance[1]

    for i, embedding in enumerate(embeddings):
        face_embedding = FaceEmbedding(face_id=index, dimension=i, value=float(embedding))
        session.add(face_embedding)

session.commit()


Processing employee 0758:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 185ms/step


Processing employee 0767:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 39ms/step


Processing employee 0760:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processing employee 0765:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 41ms/step


Processing employee 0762:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 41ms/step


100%|██████████| 13/13 [00:00<00:00, 607.73it/s]


In [41]:
# Load target image and preprocess
target_img = tf.keras.preprocessing.image.load_img("/Users/ds_learner16/Documents/Face.io/photos/val/0757/0757_test_19.jpg", target_size=(160, 160))
target_x = image.img_to_array(target_img)
target_x = np.expand_dims(target_x, axis=0)
target_x = preprocess_input(target_x)

# Get embedding for target image
target_embedding = model.predict(target_x)[0]

# Query the database for face embeddings
face_meta = session.query(FaceMeta).all()

# Find closest match in database
min_distance = float('inf')
match_employee_no = None
match_img_name = None

for face in face_meta:
    employee_embeddings = np.frombuffer(face.embedding, dtype=np.float32).reshape((-1, 128))

    # Calculate distances between target embedding and employee embeddings
    distances = np.linalg.norm(employee_embeddings - target_embedding, axis=1)

    # Find closest match
    closest_distance = np.min(distances)
    if closest_distance < min_distance:
        min_distance = closest_distance
        match_employee_no = face.empl_no
        match_img_name = face.img_name

# Print closest match
if match_employee_no is not None:
    print("Closest match found for employee", match_employee_no, "with image", match_img_name)
else:
    print("No match found in the database.")


1/1 [==============================] - 0s 141ms/step
Closest match found for employee 757 with image 


In [44]:
import cv2

# Load face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

# Define function to preprocess image
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Crop face and resize to (160, 160)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = np.expand_dims(face, axis=0)
        face = preprocess_input(face)
        return face
    else:
        return None

# Open camera and start capturing frames
cap = cv2.VideoCapture(0)

while True:
    # Capture frame from camera
    ret, frame = cap.read()

    # Preprocess image
    face = preprocess_image(frame)

    # Make prediction if face is detected
    if face is not None:
        # Get embedding for face
        embedding = model.predict(face)[0]

        # Query the database for face embeddings
        face_meta = session.query(FaceMeta).all()

        # Find closest match in database
        min_distance = float('inf')
        match_employee_no = None
        match_img_name = None

        for face in face_meta:
            employee_embeddings = np.frombuffer(face.embedding, dtype=np.float32).reshape((-1, 128))

            # Calculate distances between target embedding and employee embeddings
            distances = np.linalg.norm(employee_embeddings - embedding, axis=1)

            # Find closest match
            closest_distance = np.min(distances)
            if closest_distance < min_distance:
                min_distance = closest_distance
                match_employee_no = face.empl_no
                match_img_name = face.img_name

        # Print closest match
        if match_employee_no is not None:
            cv2.putText(frame, "Match found for employee {}".format(match_employee_no), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "No match found in database", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display frame
    cv2.imshow('Face Recognition', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break

# Release camera and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 49ms/step


In [4]:
!pip install --no-deps deepface

  Using cached deepface-0.0.79-py3-none-any.whl (49 kB)


In [29]:
!pip install cosine_similarity

In [6]:
!pip install gdown

  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepface 0.0.79 requires fire>=0.4.0, which is not installed.
deepface 0.0.79 requires gunicorn>=20.1.0, which is not installed.
deepface 0.0.79 requires mtcnn>=0.1.0, which is not installed.
deepface 0.0.79 requires retina-face>=0.0.1, which is not installed.
deepface 0.0.79 requires tensorflow>=1.9.0, which is not installed.
